# Computer Lab 2 - ENGN4528 - 2021

**Name:** Thao Pham  
**Student ID:** u7205329  
**Date:** 02/05/2021

In [ ]:
# import libraries

import numpy as np
import cv2 
import matplotlib.pyplot as plt
from os import listdir

from time import process_time

# Task 1 - Harris Corner Detector

In [ ]:
def conv2(img, conv_filter):
    # flip the filter

    f_siz_1, f_size_2 = conv_filter.shape
    conv_filter = conv_filter[range(f_siz_1 - 1, -1, -1), :][:, range(f_siz_1 - 1, -1, -1)]
    pad = (conv_filter.shape[0] - 1) // 2
    result = np.zeros((img.shape))
    img = np.pad(img, ((pad, pad), (pad, pad)), 'constant', constant_values=(0, 0))

    filter_size = conv_filter.shape[0]
    for r in np.arange(img.shape[0] - filter_size + 1):
        for c in np.arange(img.shape[1] - filter_size + 1):
            curr_region = img[r:r + filter_size, c:c + filter_size]
            curr_result = curr_region * conv_filter
            conv_sum = np.sum(curr_result)  # Summing the result of multiplication.
            result[r, c] = conv_sum  # Saving the summation in the convolution layer feature map.

    return result

In [ ]:
def fspecial(shape=(3, 3), sigma=0.5):
    m, n = [(ss - 1.) / 2. for ss in shape]
    y, x = np.ogrid[-m:m + 1, -n:n + 1]
    h = np.exp(-(x * x + y * y) / (2. * sigma * sigma))
    h[h < np.finfo(h.dtype).eps * h.max()] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

In [ ]:
# Parameters, add more if needed
thresh = 0.01

# Calculate image derivatives 

def calculate_derivs(img):
    dx = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    dy = dx.transpose()
    
    Ix = conv2(img, dx)
    Iy = conv2(img, dy)
    
    # This line underneath is to create a Gaussian 2D distribution
    # of size 13 x 13
    sigma = 2
    
    g = fspecial((max(1, np.floor(3 * sigma) * 2 + 1), max(1, np.floor(3 * sigma) * 2 + 1)), sigma)
    Iy2 = conv2(np.power(Iy, 2), g)
    Ix2 = conv2(np.power(Ix, 2), g)
    Ixy = conv2(Ix * Iy, g)
    
    return Ix2, Iy2, Ixy

Complete the missing parts, rewrite them to ‘harris.py’ as a python script, and design appropriate function signature (1 mark).

In [ ]:
######################################################################
# Task: Compute the Harris Cornerness
######################################################################

def compute_Harris_cornerness(img, alpha, Ix2, Iy2, Ixy):
    h,w = img.shape

    Rmatrix = np.zeros(img.shape)
    window = fspecial((5,5), 1)

    for i in range(2,w-2):
        for j in range(2,h-2):
            
            # Find the second-order matrix of each pixel
            Ix2_window = np.multiply(Ix2[j-2:j+3, i-2:i+3], window)
            Iy2_window = np.multiply(Iy2[j-2:j+3, i-2:i+3], window)
            Ixy_window = np.multiply(Ixy[j-2:j+3, i-2:i+3], window)

            sum_Ix2 = np.sum(Ix2_window)
            sum_Iy2 = np.sum(Iy2_window)
            sum_Ixy = np.sum(Ixy_window)
            
            M = np.array([[sum_Ix2, sum_Ixy], 
                          [sum_Ixy, sum_Iy2]])
            
            # Response value
            r = np.linalg.det(M) - alpha*(np.trace(M)**2)

            # Save the response value to a new matrix 
            Rmatrix[j,i] = r

    return Rmatrix

In [ ]:
######################################################################
# Task: Perform non-maximum suppression and
#       thresholding, return the N corner points
#       as an Nx2 matrix of x and y coordinates
######################################################################

def marking_corners(img, response_matrix, t, d):
    corners = {}

    h, w = img.shape
    
    # Thresholding
    for j in range(h):
        for i in range(w):
            if response_matrix[j,i] > t:
                corners[(j,i)] = response_matrix[j,i]
                
    # Sort the corners w.r.t to their response values in descending manner
    corners_sorted = sorted(corners.items(), key=lambda x: x[1], reverse=True)

    # Create the final_corners list in preparation for NMS
    final_corners = [corners_sorted[0]]

    # Non-maximum suppresion
    for potential_corner in corners_sorted:
        within_radius = False
        for chosen_corner in final_corners: 
            yp, xp = potential_corner[0]
            y_chosen, x_chosen = chosen_corner[0]
            
            #If a potential corner is within a radius of 8 of any existing chosen corners,
            # we will dismiss it
            if (np.abs(xp - x_chosen) <= d) and (np.abs(yp - y_chosen) <= d):
                within_radius = True
                break
                
        # If not in conflict with any chosen corners, we will add it to the final list of corners        
        if not within_radius:
            final_corners.append(potential_corner)

    # Return a Nx2 matrix of the pixel coordinates
    yc, xc = [], []

    for coord, Rval in final_corners:
        y,x = coord
        yc.append(y)
        xc.append(x)

    mat = np.column_stack((yc, xc))
    return mat

- Test this function on the provided four test images (Harris-[1,2,3,4].jpg, they can be downloaded from Wattle). Display your results by marking the detected corners on the input images (using circles or crosses, etc) **(0.5 mark for each image, 2 marks in total)**.

- Compare your results with that from python’s built-in function `cv2.cornerHarris()` **(0.5 mark)**, and discuss the factors that affect the performance of Harris corner detection **(1 mark)**.

In [ ]:
img_files = []
for n in range(1, 5):
    file = 'Task1/Harris_{}.jpg'.format(n)
    img = cv2.imread(file, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_files.append(img)

In [ ]:
i = 1
for img in img_files:
    fig, axs = plt.subplots(1, 2, figsize = (12, 7))

    # Own function
    Ix2, Iy2, Ixy = calculate_derivs(img)
    response_matrix = compute_Harris_cornerness(img, 0.04, Ix2, Iy2, Ixy)

    threshold = np.max(response_matrix) * 0.01
    M1 = marking_corners(img, response_matrix, threshold, 8)
    
    ys, xs = M1[:,0], M1[:, 1]
    axs[0].imshow(img, cmap = 'gray')
    axs[0].plot(xs, ys, '*', color='red', markersize = 4)
    axs[0].axis('off')

    # Built-in function
    res = cv2.cornerHarris(img, 5, 9, 0.04)
    threshold2 = np.max(res)*0.01
    M2 = marking_corners(img, res, threshold2, 8)

    ys2, xs2 = M2[:,0], M2[:, 1]
    axs[1].imshow(img, cmap = 'gray')
    axs[1].plot(xs2, ys2, '*', color='yellow', markersize = 4)
    axs[1].axis('off')

    plt.tight_layout()
    plt.savefig('Task1/results/Harris-{}-result.png'.format(i))
    plt.show()
    
    i += 1

Test this function on `Harris-5.jpg`. Analyse the results why we cannot get corners by discussing and visualising your corner response scores of the image. **(0.5 mark)**

In [ ]:
file = 'Task1/Harris-5.jpg'

img = cv2.imread(file, 1)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

Ix2, Iy2, Ixy = calculate_derivs(img)
response_matrix = compute_Harris_cornerness(img, 0.04, Ix2, Iy2, Ixy)

threshold = 0.01*np.max(response_matrix)
M = marking_corners(img, response_matrix, threshold, 8)
ys, xs = M[:,0], M[:, 1]

In [ ]:
# Display

fig, axs = plt.subplots(1, 3, figsize = (11, 4))

axs[0].imshow(img, cmap = 'gray')
axs[0].plot(xs, ys, '*', color='red', markersize = 2)
axs[1].imshow(response_matrix, cmap = 'hot')
axs[2].imshow(Iy2)

i = 0
for name in ('(a)', '(b)', '(c)'):
    axs[i].set_title(name, fontsize = 12)
    i += 1
    
plt.tight_layout()
plt.savefig('Task1/results/Harris-5-results.png')
plt.show()

Test this function on `Harris-6.jpg`. Plot your harris corner detector results in your report and propose a solution to obtain the salient corners which is robust to noise. **(0.5 mark)**

In [ ]:
file = 'Task1/Harris-6.jpg'

img = cv2.imread(file, 1)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

Ix2, Iy2, Ixy = calculate_derivs(img)
response_matrix = compute_Harris_cornerness(img, 0.04, Ix2, Iy2, Ixy)

threshold = 0.01*np.max(response_matrix)
M = marking_corners(img, response_matrix, threshold, 8)
ys, xs = M[:,0], M[:, 1]

In [ ]:
def my_gauss_filter(noisy_image, kernel):
    height, width = noisy_image.shape
    kernel_size = kernel.shape[0]
    
    half = kernel_size//2
    
    output_image = np.zeros((height, width), dtype = np.double)
    
    for i in range(height):
        for j in range(width):
            pixel = 0
            for k in range(0, kernel_size):
                for l in range(0, kernel_size):
                    try:
                        pixel += kernel[k,l]*noisy_image[i-half+k, j-half+l]
                    except IndexError: 
                    # This is to deal with border cases, when an IndexError
                    # is raised it means that we're trying to average with a pixel
                    # outside of the boundary, so we set the pixel value to 0
                        pixel += 0
            output_image[i,j] = pixel
        
    # Clipping output image
    output_image[output_image < 0] = 0
    output_image[output_image > 255] = 255
    
    return output_image.astype(np.uint8)

In [ ]:
# Apply Gaussian filter to Harris-6

img2 = my_gauss_filter(img, fspecial((7,7), 3))

In [ ]:
# Repeat the process for blurred Harris-6

Ix2, Iy2, Ixy = calculate_derivs(img2)
response_matrix = compute_Harris_cornerness(img2, 0.04, Ix2, Iy2, Ixy)

#response_matrix = cv2.cornerHarris(img, 5, 13, 0.04)
threshold = 0.01*np.max(response_matrix)
M = marking_corners(img2, response_matrix, threshold, 8)
ys2, xs2 = M[:,0], M[:, 1]

In [ ]:
# Display

fig, axs = plt.subplots(1, 2, sharey=True)

axs[0].imshow(img, cmap = 'gray')
axs[0].plot(xs, ys, '*', color='red', markersize = 2)
axs[0].set_title('(a)')
axs[1].imshow(img2, cmap = 'gray')
axs[1].plot(xs2, ys2, '*', color='red', markersize = 2)
axs[1].set_title('(b)')

plt.tight_layout()
plt.savefig('Task1/results/Harris-6-result.png')
plt.show()

# Task 2 - K-Means Clustering and Colour Image Segmentation

Implement your own K-means function `my_kmeans()`. The input is the data points to be processed and the number of clusters, and the output is several clusters of your data (you can use a cell array for clusters). Make sure each step in K-means is correct and clear, and comment on key code fragments **(1.5 marks)**.

In [ ]:
# This function sets k random data points in the data matrix to 
# be the initial k cluster centers

def initialise_parameters(X, k):
    npoints, dim = X.shape
    C = np.zeros((k, dim))
    
    chosen_index = np.random.choice(npoints, k, replace = False)

    for i in range(k):
        C[i] = X[chosen_index[i]]
     
    return C

In [ ]:
def assignment(C, X):
    L = np.zeros(X.shape)
    
    i = 0
    while i < len(X):
        data_point = X[i]
        
        # Calculate the distance from the data point to each centroid
        min_distance = 99999
        closest_rep = None
        
        for j in range(len(C)):
            current_centroid = C[j]
            dist = np.linalg.norm(data_point - current_centroid)
            if dist < min_distance:
                closest_rep = current_centroid
                min_distance = dist
        
        # Assignment 
        L[i] = closest_rep
        i += 1
    
    return L

In [ ]:
def recalculate_centroids(C, X, L):
    total = []
    count = []
    
    row, col = X.shape
    for n in range(len(C)):
        total.append(np.zeros(col))

    count = [0 for n in range(len(C))]
    
    # Calculating the total and count for each cluster
    for i in range(len(X)):
        data_point = X[i]
        
        for n in range(len(C)):
            if np.array_equal(L[i], C[n]):
                total[n] += data_point
                count[n] += 1

    # Calculating the new centroids          
    new_C = [total[n] / count[n] for n in range(len(total))]

    return np.array(new_C)

In [ ]:
def my_kmeans(X, k):
    L = np.zeros(X.shape)
    C = initialise_parameters(X, k)
    
    niter = 0
    i = 100 
    old_C = C
    
    while niter < i:
        L = assignment(C, X)
        C = recalculate_centroids(C, X, L)

        norm = np.linalg.norm(old_C - C, axis = 1)
        stop = np.any(norm < 0.2)
        if stop: 
            return L

        old_C = C
        niter += 1

    return L

Apply your K-means function to color image segmentation. Each pixel should be represented as a 5-D vector that encodes: 
1. $L^∗$ - lightness of the color; 
2. $a^∗$ - the color position between red and green; 
3. $b^∗$ - the position between yellow and blue; 
4. x, y - pixel coordinates.

In [ ]:
# This function is used to display results

def display_result(original_img, Z, res):
    mat = np.zeros((original_img.shape))

    for n in range(len(Z)):
        row = Z[n,:]
        row_class = res[n,:]
        xc, yc = row[-2], row[-1]

        mat[int(yc), int(xc)] = row_class[:3]

    return cv2.cvtColor(mat.astype('uint8'), cv2.COLOR_Lab2RGB)

## M&M Picture

In [ ]:
# Prepare data

mm_og = cv2.imread('Task2/mandm.png', 1)
mm_lab = cv2.cvtColor(mm_og, cv2.COLOR_BGR2Lab)

# 5d (with pixel coordinates)

mm = mm_lab.astype('float32')
X = []

height, width = mm.shape[:2]
for i in range(width):
    for j in range(height):
        att = []
        for each in mm[j, i, :]:
            att.append(each)
        att.append(i)
        att.append(j)
        X.append(att)
        
mm_5d = np.array(X)

# 3d (without pixel coordinates)
mm_3d = mm.reshape(-1, 3)

### Compare segmentation results using different numbers of clusters

In [ ]:
label7 = my_kmeans(mm_5d, 7)
segmented_image7 = display_result(mm_og, mm_5d, label7)

In [ ]:
label20 = my_kmeans(mm_5d, 20)
segmented_image20 = display_result(mm_og, mm_5d, label20)

label40 = my_kmeans(mm_5d, 40)
segmented_image40 = display_result(mm_og, mm_5d, label40)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize = (10, 3))
axs[0].imshow(segmented_image7)
axs[1].imshow(segmented_image20)
axs[2].imshow(segmented_image40)

k_ls = [7, 20, 40]
n = 0
for ax in axs.flat:
    ax.axis('off')
    ax.set_title('k = {}'.format(k_ls[n]))
    n += 1
    
plt.tight_layout()
plt.savefig('Task2/results/Task2-M&M-5D.png')
plt.show()

### Compare segmentation results with and without pixel coordinates

In [ ]:
h,w,c = mm_og.shape

label7_3d = my_kmeans(mm_3d, 7)
segmented7_3d = label7_3d.reshape(h,w,c)
segmented7_3d_rgb = cv2.cvtColor(segmented7_3d.astype('uint8'), cv2.COLOR_Lab2RGB)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize =(6,3))

axs[0].imshow(segmented_image7)
axs[0].set_title('With pixel')
axs[1].imshow(segmented7_3d_rgb)
axs[1].set_title('Without pixel')

for ax in axs:
    ax.axis('off')
plt.tight_layout()
#plt.savefig('Task2/results/Task2-M&M-3D.png')
plt.show()

## Pepper picture 

In [ ]:
fruit_og = cv2.imread('Task2/peppers.png', 1)
fruit_lab = cv2.cvtColor(fruit_og, cv2.COLOR_BGR2Lab)

fruit = fruit_lab.astype('float')
X_fruit = []

height, width = fruit.shape[:2]
for i in range(width):
    for j in range(height):
        att = []
        for each in fruit[j, i, :]:
            att.append(each)
        att.append(i)
        att.append(j)
        X_fruit.append(att)
        
fruit_5d = np.array(X_fruit)

# 3d (without pixel coordinates)
fruit_3d = fruit.reshape(-1, 3)

### Compare segmentation results using different numbers of clusters

In [ ]:
label7 = my_kmeans(fruit_5d, 7)
segmented_image7 = display_result(fruit_og, fruit_5d, label7)

In [ ]:
label20 = my_kmeans(fruit_5d, 20)
segmented_image20 = display_result(fruit_og, fruit_5d, label20)

label40 = my_kmeans(fruit_5d, 40)
segmented_image40 = display_result(fruit_og, fruit_5d, label40)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize = (10, 3))
axs[0].imshow(segmented_image7)
axs[1].imshow(segmented_image20)
axs[2].imshow(segmented_image40)

k_ls = [7, 20, 40]
n = 0
for ax in axs.flat:
    ax.axis('off')
    ax.set_title('k = {}'.format(k_ls[n]))
    n += 1
    
plt.tight_layout()
plt.savefig('Task2/results/Task2-Fruit-5D.png')
plt.show()

### Compare segmentation results with and without pixel coordinates

In [ ]:
h,w,c = fruit_og.shape

label7_3d = my_kmeans(fruit_3d, 7)
segmented7_3d = label7_3d.reshape(h,w,c)
segmented7_3d_rgb = cv2.cvtColor(segmented7_3d.astype('uint8'), cv2.COLOR_Lab2RGB)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (7,3))

axs[0].imshow(segmented_image7)
axs[0].set_title('With pixel')
axs[1].imshow(segmented7_3d_rgb)
axs[1].set_title('Without pixel')

for ax in axs:
    ax.axis('off')
plt.tight_layout()
plt.savefig('Task2/results/Task2-Fruit-3D.png')
plt.show()

## K-means++

The standard K-means algorithm is sensitive to initialization (e.g. initial cluster centres/seeds). One possible solution is to use `K-means++`, in which the initial seeds are forced to be far away from each other (to avoid local minimum). 

Please read the material http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf, summarize the key steps in the report (0.5 mark), and then implement K-means++ in your standard algorithm as a new initialization strategy (0.5 mark). 

Compare the image segmentation performance (e.g., in terms of convergence speed and segmentation results [by plotting the segmentation results in the report]) of this new strategy, with that of standard K-means, using different numbers of clusters and the same 5-D point representation, *namely [L, a*, b*, x, y] (L, a, b , pixel coordinates)* as that from previous question (0.5 mark).

In [ ]:
# TODO: K-means++

def initialise_parameters_Kplus(X, k):
    npoints, dim = X.shape
    C = np.zeros((k, dim))
    chosen_index = np.zeros(k)
    
    # Choose the first random centroid by select a random row
    chosen_centroid = X[np.random.randint(0, npoints)]
    C[0] = chosen_centroid
    
    # Use kmeans++ to find the other centroids
    i = 0
    while i < k-1:
        new_X_index = []
        Dx_list = []
        
        for j in range(len(X)):
            data_point = X[j]
            
            if data_point not in C:
                new_X_index.append(j)
                
                Dx = float('inf')
                for centroid in C:
                    distance = np.linalg.norm(data_point - centroid)
                    Dx = min(Dx, distance)
            
                Dx_list.append(Dx**2)
               
        # Compute the new probability distribution     
        sum_Dx = np.sum(Dx_list)
        probs = np.array(Dx_list) / sum_Dx
        
        # Choose a new centroid
        new_centroid_index = np.random.choice(new_X_index, 1, p = probs)
        C[i+1] = X[new_centroid_index[0]]
        i += 1
        
    return C

In [ ]:
# TODO: Change my_kmeans function to specify method of initialisation

def my_kmeans_changed(X, k, init):
    L = np.zeros(X.shape)
    
    if init == 'plus':
        C = initialise_parameters_Kplus(X, k)
    elif init == 'normal':
        C = initialise_parameters(X, k)
    
    tstart = process_time()
    
    niter = 0
    i = 100 
    old_C = C
    
    while niter < i:
        L = assignment(C, X)
        C = recalculate_centroids(C, X, L)

        norm = np.linalg.norm(old_C - C, axis = 1)
        stop = np.any(norm < 0.8)
        if stop: 
            tstop = process_time()
    
            if init == 'plus':
                print('Elapsed time with Kmeans++: {}'.format(tstop - tstart))
            elif init == 'normal':
                print('Elapsed time: {}'.format(tstop - tstart))
            
            return L

        old_C = C
        niter += 1
        
    tstop = process_time()
    
    if init == 'plus':
        print('Elapsed time with Kmeans++: {}'.format(tstop - tstart))
    elif init == 'normal':
        print('Elapsed time: {}'.format(tstop - tstart))

    return L

In [ ]:
# TODO: Compare running time

label20_normal = my_kmeans_changed(fruit_5d, 20, 'normal')
segmented_image20_normal = display_result(fruit_og, fruit_5d, label20_normal)

In [ ]:
label20_plus = my_kmeans_changed(fruit_5d, 20, 'plus')
segmented_image20_plus = display_result(fruit_og, fruit_5d, label20_plus)

In [ ]:
# TODO: Compare running time

label40_normal = my_kmeans_changed(fruit_5d, 40, 'normal')
segmented_image40_normal = display_result(fruit_og, fruit_5d, label40_normal)

label40_plus = my_kmeans_changed(fruit_5d, 40, 'plus')
segmented_image40_plus = display_result(fruit_og, fruit_5d, label40_plus)

In [ ]:
fig, axs = plt.subplots(3, 3, constrained_layout = True,
                       gridspec_kw = {'width_ratios': [1,7,7], 'height_ratios': [1,7,7]},
                       figsize = (10, 7))

axs[1][0].text(0.5, 0.5, 'k = 20', fontsize = 12)
axs[2][0].text(0.5, 0.5, 'k = 40', fontsize = 12)
axs[0][1].text(0.3, 0.5, 'Random initialisation', fontsize = 12)
axs[0][2].text(0.4, 0.5, 'K-means++', fontsize = 12)

axs[1][1].imshow(segmented_image20_normal)
axs[2][1].imshow(segmented_image40_normal)

axs[1][2].imshow(segmented_image20_plus)
axs[2][2].imshow(segmented_image40_plus)

for ax in axs.flatten():
    ax.axis('off')
    
plt.savefig('Task2/results/Task2-KmeansPlus.png')    
plt.show()

# Task 3 - Face Recognition using Eigenface

Train an Eigen-face recognition system. Specifically, at least your face recognition system should be able to complete the following tasks:

Read all the 135 training images from Yale-Face, represent each image as a single data point in a high dimensional space and collect all the data points into a big data matrix.

In [ ]:
# TODO: Create data matrix

path_wo_me = 'Task3/Yale-FaceA/trainingset'

def make_matrix(path):
    files = listdir(path)
    files = [elem for elem in files if elem.endswith('.png')]
    
    
    img0 = cv2.imread("{}/{}".format(path, files[0]), 0)
    h, w = img0.shape
    
    A = img0.flatten()
    
    for i in range(1, len(files)):
        fname = "{}/{}".format(path, files[i])
        img = cv2.imread(fname, 0)
        img = img.flatten()
        A = np.column_stack((A, img))
        
    return files, A.astype('float')

files, A_wo_me = make_matrix(path_wo_me)
h, w = 231, 195

Perform PCA on the data matrix (1 mark), and display the mean face (1 mark). 

Given the size and the large number of input images, directly performing eigen value decomposition of the covariance matrix would be slow. Please read lecture notes and find a faster way to compute eigen values and vectors, explain the reason (1 mark) and implement it in your own code (1 mark).

In [ ]:
# TODO: Display the mean face

def make_mean_vector(data_matrix):
    return np.mean(data_matrix, axis = 1)

mean_vector = make_mean_vector(A_wo_me)
mean_image = mean_vector.reshape(h,w)
plt.imshow(mean_image.astype('uint8'), cmap = 'gray')
plt.axis('off')
plt.savefig('Task3/results/mean_face.png')
plt.show()

In [ ]:
# TODO: Perform PCA, 
# the function takes the data matrix, mean vector, and the number of 
# eigenfaces to choose

def get_eigenfaces(data_matrix, mean_vector, k):
    nfiles = data_matrix.shape[1]
    
    # Repeat the mean vector 
    mean_matrix = np.tile(mean_vector, (nfiles, 1))
    
    # Centered the data 
    A_centred = data_matrix - mean_matrix.T
    
    # Decompose A^T A
    cov = (A_centred.T @ A_centred) 
    eigvals, eigvecs = np.linalg.eig(cov)
    
    # Get u = Av
    u = np.array([A_centred@np.real(eigvecs[:,i]) for i in range(k)]).T
    
    # Normalised u
    u_normalised = np.array([u[:,i] / np.linalg.norm(u[:,i]) for i in range(k)])
    return u_normalised.T

Determine the top k principal components and visualize the top-k eigenfaces in your report (1 mark). You can choose k = 10 or k = 15.

In [ ]:
# TODO: Visualise top 15 eigenfaces

F = get_eigenfaces(A_wo_me, mean_vector, 15)


fig, axs = plt.subplots(3, 5, figsize = (10, 7))
n = 0
for i in range(3):
    for j in range(5):
        each = F[:,n]
        each = each.reshape(h,w)
        axs[i,j].imshow(each, cmap = 'gray')
        axs[i,j].axis('off')
        n += 1
plt.tight_layout()
plt.savefig('Task3/results/Eigenfaces.png')
plt.show()

For each of the 10 test images in the Yale-Face dataset, please read in an image as the reference one, and determine its projection onto the basis spanned by the top k eigenfaces. Use this projection as feature to perform a `nearest-neighbour` search over all 135 faces, and find out the top three face images that are most similar to the reference one.

In [ ]:
# TODO Reduce Dimensionality 
# Takes in an image, the main k eigenfaces, mean vector
# to project the img to a lower-dimensional subspace

def reduce_reconstruct(img_path, F, mean_vector):
    img = cv2.imread(img_path, 0)
    img_vector = img.flatten()
    
    h,w = img.shape
    
    reconstruct = np.zeros((h*w))
    dim, k = F.shape

    for i in range(k):
        eigface = F[:,i]
        subtract = img_vector.astype("float") - mean_vector
        coeff = np.dot(eigface, subtract)
        reconstruct += coeff*eigface
        
    result = np.add(reconstruct,mean_vector)
    result = np.clip(result, 0, 255)
    return result

In [ ]:
compressed_train_faces = np.zeros(A_wo_me.shape)

i = 0 
for each in files:
    training_file = '{}/{}'.format(path_wo_me, each)
    reduce_each = reduce_reconstruct(training_file, F, mean_vector)
    compressed_train_faces[:,i] = reduce_each
    i+=1

In [ ]:
# TODO: Nearest Neighbour

def nearest_images(test_image, compressed_train_faces, A):
    compressed_test_image = reduce_reconstruct(test_image, F, mean_vector)
    dim, npic = compressed_train_faces.shape
    
    distance_ls = []
    for i in range(npic):
        distance = np.linalg.norm(compressed_train_faces[:,i] - compressed_test_image)
        distance_ls.append((distance, i))
        
    distance_ls = sorted(distance_ls)
    
    chosen_3files = np.zeros((dim, 3))
    
    for i in range(3):
        
        chosen_index = distance_ls[i][1]
        chosen_3files[:,i] = A[:,chosen_index]
    
    return chosen_3files

In [ ]:
testfiles = listdir('Task3/Yale-FaceA/testset')
testfiles = [elem for elem in testfiles if elem.endswith('.png')]

for i in range(len(testfiles)):
    test_filepath = 'Task3/Yale-FaceA/testset/' + testfiles[i]

    res = nearest_images(test_filepath, compressed_train_faces, A_wo_me)
    
    test_img = cv2.imread(test_filepath, 0)
    fig, axs = plt.subplots(1, 4)
    axs[0].imshow(test_img, cmap = 'gray')
    axs[0].axis('off')
    for i in range(1, 1+3):
        axs[i].imshow(res[:,i-1].reshape(h,w), cmap = 'gray')
        axs[i].axis('off')

    plt.show()

Read in one of your own frontal face images. Then run your face recognition system on this new image. Display the top 3 faces in the training folder that are most similar to your own face (1 mark).

In [ ]:
# TODO: Display

me_test_path = 'Task3/Yale-FaceA/me-3-9.png'
me_test_img = cv2.imread(me_test_path, 0)

closest_3imgs_me1 = nearest_images(me_test_path, 
                                   compressed_train_faces, A_wo_me)

fig, axs = plt.subplots(1, 4, figsize = (8,3))

axs[0].imshow(me_test_img, cmap = 'gray')

for i in range(1, 1+3):
    nb = closest_3imgs_me1[:,i-1].reshape(h,w)
    axs[i].imshow(nb, cmap = 'gray') 
    
for ax in axs:
    ax.axis('off')
plt.tight_layout()
plt.savefig('Task3/results/me.png')
plt.show()

Repeat the previous experiment by pre-adding the other 9 additional images of your face into the training set (a total of 144 training images).

Note that you should make sure that your test face image is different from those included in the training set. Display the top 3 faces that are the closest to your face (1 mark).

In [ ]:
# Repeat the process with my images in the training set

path_with_me = 'Task3/Yale-FaceA/trainingset-with-me'
files, A_with_me = make_matrix(path_with_me)
mean_vector = make_mean_vector(A_with_me)
F = get_eigenfaces(A_with_me, mean_vector, 15)

compressed_train_faces = np.zeros(A_with_me.shape)

i = 0 
for each in files:
    training_file = path_with_me + '/' + each
    reduce_each = reduce_reconstruct(training_file, F, mean_vector)
    compressed_train_faces[:,i] = reduce_each
    i+=1
    
closest_3imgs_me1 = nearest_images(me_test_path, compressed_train_faces, A_with_me)

fig, axs = plt.subplots(1, 4, figsize = (8,3))

axs[0].imshow(me_test_img, cmap = 'gray')

for i in range(1, 1+3):
    nb = closest_3imgs_me1[:,i-1].reshape(h,w)
    axs[i].imshow(nb, cmap = 'gray') 
    
for ax in axs:
    ax.axis('off')
plt.tight_layout()
plt.savefig('Task3/results/me2.png')
plt.show()